In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Heart Disease Prediction - Model Experimentation\n",
    "**Team**: Mercy Thokozani Ngwenya & Mediator Nhongo  \n",
    "**Date**: $(new Date().toLocaleDateString())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Machine Learning imports\n",
    "from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV\n",
    "from sklearn.preprocessing import StandardScaler, LabelEncoder\n",
    "from sklearn.impute import SimpleImputer\n",
    "from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier\n",
    "from sklearn.svm import SVC\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.tree import DecisionTreeClassifier\n",
    "from sklearn.metrics import (\n",
    "    accuracy_score, precision_score, recall_score, f1_score, \n",
    "    roc_auc_score, confusion_matrix, classification_report,\n",
    "    roc_curve, precision_recall_curve\n",
    ")\n",
    "\n",
    "# Set style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline\n",
    "\n",
    "print(\"All packages imported successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Loading and Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load dataset\n",
    "column_names = [\n",
    "    \"age\", \"sex\", \"cp\", \"trestbps\", \"chol\", \"fbs\", \"restecg\",\n",
    "    \"thalach\", \"exang\", \"oldpeak\", \"slope\", \"ca\", \"thal\", \"num\"\n",
    "]\n",
    "\n",
    "df = pd.read_csv('../data/cleveland.data', names=column_names, na_values='?', skipinitialspace=True)\n",
    "\n",
    "print(\"Dataset loaded successfully!\")\n",
    "print(f\"Original shape: {df.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create binary target\n",
    "df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)\n",
    "\n",
    "# Separate features and target\n",
    "X = df.drop(columns=['num', 'target'])\n",
    "y = df['target']\n",
    "\n",
    "print(f\"Features shape: {X.shape}\")\n",
    "print(f\"Target distribution:\\n{y.value_counts()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Handle missing values\n",
    "print(\"Missing values before handling:\")\n",
    "print(X.isnull().sum())\n",
    "\n",
    "# Impute numerical features\n",
    "numerical_cols = X.select_dtypes(include=[np.number]).columns\n",
    "categorical_cols = X.select_dtypes(include=['object']).columns\n",
    "\n",
    "imputer = SimpleImputer(strategy='median')\n",
    "X[numerical_cols] = imputer.fit_transform(X[numerical_cols])\n",
    "\n",
    "# For categorical, use mode\n",
    "for col in categorical_cols:\n",
    "    if X[col].isnull().any():\n",
    "        mode_val = X[col].mode()[0]\n",
    "        X[col].fillna(mode_val, inplace=True)\n",
    "\n",
    "print(\"\\nMissing values after handling:\")\n",
    "print(X.isnull().sum().sum(), \"missing values remaining\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Encode categorical variables\n",
    "label_encoders = {}\n",
    "for col in categorical_cols:\n",
    "    label_encoders[col] = LabelEncoder()\n",
    "    X[col] = label_encoders[col].fit_transform(X[col].astype(str))\n",
    "\n",
    "# Scale numerical features\n",
    "scaler = StandardScaler()\n",
    "X[numerical_cols] = scaler.fit_transform(X[numerical_cols])\n",
    "\n",
    "print(\"Preprocessing completed!\")\n",
    "print(f\"Final feature matrix shape: {X.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Split data\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y, test_size=0.2, random_state=42, stratify=y\n",
    ")\n",
    "\n",
    "print(\"Data split completed:\")\n",
    "print(f\"Training set: {X_train.shape}\")\n",
    "print(f\"Test set: {X_test.shape}\")\n",
    "print(f\"Train target distribution:\\n{y_train.value_counts()}\")\n",
    "print(f\"Test target distribution:\\n{y_test.value_counts()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Model Definitions and Training"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define models and their hyperparameters\n",
    "models = {\n",
    "    'Logistic Regression': {\n",
    "        'model': LogisticRegression(random_state=42, max_iter=1000),\n",
    "        'params': {\n",
    "            'C': [0.1, 1, 10],\n",
    "            'solver': ['liblinear', 'lbfgs']\n",
    "        }\n",
    "    },\n",
    "    'Random Forest': {\n",
    "        'model': RandomForestClassifier(random_state=42),\n",
    "        'params': {\n",
    "            'n_estimators': [100, 200],\n",
    "            'max_depth': [10, 20, None],\n",
    "            'min_samples_split': [2, 5]\n",
    "        }\n",
    "    },\n",
    "    'Gradient Boosting': {\n",
    "        'model': GradientBoostingClassifier(random_state=42),\n",
    "        'params': {\n",
    "            'n_estimators': [100, 200],\n",
    "            'learning_rate': [0.05, 0.1],\n",
    "            'max_depth': [3, 5]\n",
    "        }\n",
    "    },\n",
    "    'SVM': {\n",
    "        'model': SVC(random_state=42, probability=True),\n",
    "        'params': {\n",
    "            'C': [0.1, 1, 10],\n",
    "            'kernel': ['linear', 'rbf']\n",
    "        }\n",
    "    },\n",
    "    'Decision Tree': {\n",
    "        'model': DecisionTreeClassifier(random_state=42),\n",
    "        'params': {\n",
    "            'max_depth': [5, 10, 20],\n",
    "            'min_samples_split': [2, 5, 10]\n",
    "        }\n",
    "    }\n",
    "}\n",
    "\n",
    "print(\"Models defined for experimentation:\")\n",
    "for name in models.keys():\n",
    "    print(f\"- {name}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train models with GridSearchCV\n",
    "results = {}\n",
    "best_models = {}\n",
    "\n",
    "for name, config in models.items():\n",
    "    print(f\"\\nTraining {name}...\")\n",
    "    \n",
    "    # Perform grid search\n",
    "    grid_search = GridSearchCV(\n",
    "        config['model'], \n",
    "        config['params'], \n",
    "        cv=5, \n",
    "        scoring='accuracy',\n",
    "        n_jobs=-1,\n",
    "        verbose=0\n",
    "    )\n",
    "    \n",
    "    grid_search.fit(X_train, y_train)\n",
    "    \n",
    "    # Store results\n",
    "    best_models[name] = grid_search.best_estimator_\n",
    "    results[name] = {\n",
    "        'best_score': grid_search.best_score_,\n",
    "        'best_params': grid_search.best_params_,\n",
    "        'best_estimator': grid_search.best_estimator_\n",
    "    }\n",
    "    \n",
    "    print(f\"{name} - Best CV Score: {grid_search.best_score_:.4f}\")\n",
    "    print(f\"Best parameters: {grid_search.best_params_}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Model Evaluation and Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate models on test set\n",
    "evaluation_results = {}\n",
    "\n",
    "for name, model in best_models.items():\n",
    "    # Predictions\n",
    "    y_pred = model.predict(X_test)\n",
    "    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None\n",
    "    \n",
    "    # Calculate metrics\n",
    "    metrics = {\n",
    "        'accuracy': accuracy_score(y_test, y_pred),\n",
    "        'precision': precision_score(y_test, y_pred, average='weighted'),\n",
    "        'recall': recall_score(y_test, y_pred, average='weighted'),\n",
    "        'f1_score': f1_score(y_test, y_pred, average='weighted'),\n",
    "        'cv_score': results[name]['best_score']\n",
    "    }\n",
    "    \n",
    "    if y_pred_proba is not None:\n",
    "        metrics['roc_auc'] = roc_auc_score(y_test, y_pred_proba)\n",
    "    \n",
    "    evaluation_results[name] = metrics\n",
    "    \n",
    "    print(f\"\\n{name} Test Results:\")\n",
    "    print(f\"Accuracy: {metrics['accuracy']:.4f}\")\n",
    "    print(f\"Precision: {metrics['precision']:.4f}\")\n",
    "    print(f\"Recall: {metrics['recall']:.4f}\")\n",
    "    print(f\"F1-Score: {metrics['f1_score']:.4f}\")\n",
    "    if 'roc_auc' in metrics:\n",
    "        print(f\"ROC AUC: {metrics['roc_auc']:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comparison DataFrame\n",
    "comparison_df = pd.DataFrame(evaluation_results).T\n",
    "comparison_df = comparison_df.sort_values('accuracy', ascending=False)\n",
    "\n",
    "print(\"\\n=== MODEL PERFORMANCE COMPARISON ===\")\n",
    "comparison_df"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize model comparison\n",
    "plt.figure(figsize=(14, 8))\n",
    "\n",
    "metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1_score']\n",
    "x_pos = np.arange(len(comparison_df))\n",
    "bar_width = 0.2\n",
    "\n",
    "for i, metric in enumerate(metrics_to_plot):\n",
    "    plt.bar(x_pos + i * bar_width, comparison_df[metric], \n",
    "            width=bar_width, label=metric.replace('_', ' ').title())\n",
    "\n",
    "plt.xlabel('Models')\n",
    "plt.ylabel('Score')\n",
    "plt.title('Model Performance Comparison', fontsize=16, fontweight='bold')\n",
    "plt.xticks(x_pos + bar_width * 1.5, comparison_df.index, rotation=45)\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.ylim(0, 1)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Detailed Analysis of Best Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Identify best model\n",
    "best_model_name = comparison_df.index[0]\n",
    "best_model = best_models[best_model_name]\n",
    "\n",
    "print(f\"\\n=== BEST MODEL: {best_model_name} ===\")\n",
    "print(f\"Test Accuracy: {comparison_df.loc[best_model_name, 'accuracy']:.4f}\")\n",
    "print(f\"CV Score: {comparison_df.loc[best_model_name, 'cv_score']:.4f}\")\n",
    "print(f\"Best Parameters: {results[best_model_name]['best_params']}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Detailed evaluation of best model\n",
    "best_predictions = best_model.predict(X_test)\n",
    "best_probabilities = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, 'predict_proba') else None\n",
    "\n",
    "print(\"=== DETAILED CLASSIFICATION REPORT ===\")\n",
    "print(classification_report(y_test, best_predictions, \n",
    "                          target_names=['No Disease', 'Disease']))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Confusion Matrix\n",
    "plt.figure(figsize=(10, 8))\n",
    "cm = confusion_matrix(y_test, best_predictions)\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', \n",
    "            xticklabels=['No Disease', 'Disease'],\n",
    "            yticklabels=['No Disease', 'Disease'])\n",
    "plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')\n",
    "plt.ylabel('Actual')\n",
    "plt.xlabel('Predicted')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ROC Curve\n",
    "if best_probabilities is not None:\n",
    "    plt.figure(figsize=(10, 8))\n",
    "    \n",
    "    fpr, tpr, _ = roc_curve(y_test, best_probabilities)\n",
    "    roc_auc = roc_auc_score(y_test, best_probabilities)\n",
    "    \n",
    "    plt.plot(fpr, tpr, color='darkorange', lw=2, \n",
    "             label=f'ROC curve (AUC = {roc_auc:.4f})')\n",
    "    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')\n",
    "    plt.xlim([0.0, 1.0])\n",
    "    plt.ylim([0.0, 1.05])\n",
    "    plt.xlabel('False Positive Rate')\n",
    "    plt.ylabel('True Positive Rate')\n",
    "    plt.title(f'ROC Curve - {best_model_name}', fontsize=14, fontweight='bold')\n",
    "    plt.legend(loc='lower right')\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Feature Importance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature importance for tree-based models\n",
    "if hasattr(best_model, 'feature_importances_'):\n",
    "    feature_importance = pd.DataFrame({\n",
    "        'feature': X.columns,\n",
    "        'importance': best_model.feature_importances_\n",
    "    }).sort_values('importance', ascending=False)\n",
    "    \n",
    "    print(\"=== FEATURE IMPORTANCE ===\")\n",
    "    print(feature_importance.head(10))\n",
    "    \n",
    "    # Plot feature importance\n",
    "    plt.figure(figsize=(12, 8))\n",
    "    top_features = feature_importance.head(10)\n",
    "    \n",
    "    sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')\n",
    "    plt.title(f'Top 10 Feature Importances - {best_model_name}', \n",
    "              fontsize=16, fontweight='bold')\n",
    "    plt.xlabel('Importance Score')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(f\"{best_model_name} does not support feature importance visualization\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Cross-Validation Stability Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cross-validation scores for all models\n",
    "cv_results = {}\n",
    "\n",
    "for name, model_config in models.items():\n",
    "    model = model_config['model']\n",
    "    # Set best parameters\n",
    "    model.set_params(**results[name]['best_params'])\n",
    "    \n",
    "    # Perform cross-validation\n",
    "    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')\n",
    "    cv_results[name] = cv_scores\n",
    "    \n",
    "    print(f\"{name} - CV Scores: {cv_scores}\")\n",
    "    print(f\"{name} - Mean CV: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize CV stability\n",
    "plt.figure(figsize=(14, 8))\n",
    "\n",
    "box_data = [cv_results[name] for name in cv_results.keys()]\n",
    "box_plot = plt.boxplot(box_data, labels=cv_results.keys(), patch_artist=True)\n",
    "\n",
    "# Color the boxes\n",
    "colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightsalmon', 'lightyellow']\n",
    "for patch, color in zip(box_plot['boxes'], colors):\n",
    "    patch.set_facecolor(color)\n",
    "\n",
    "plt.title('Cross-Validation Score Distribution', fontsize=16, fontweight='bold')\n",
    "plt.ylabel('Accuracy Score')\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Model Interpretation and Business Insights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== BUSINESS AND CLINICAL INSIGHTS ===\")\n",
    "\n",
    "# Model performance summary\n",
    "best_accuracy = comparison_df.loc[best_model_name, 'accuracy']\n",
    "best_precision = comparison_df.loc[best_model_name, 'precision']\n",
    "best_recall = comparison_df.loc[best_model_name, 'recall']\n",
    "\n",
    "print(f\"1. Best Model: {best_model_name}\")\n",
    "print(f\"2. Overall Accuracy: {best_accuracy:.1%}\")\n",
    "print(f\"3. Precision (Correct Disease Predictions): {best_precision:.1%}\")\n",
    "print(f\"4. Recall (Disease Cases Identified): {best_recall:.1%}\")\n",
    "\n",
    "# Clinical implications\n",
    "print(\"\\n5. Clinical Implications:\")\n",
    "print(\"   - High recall is crucial to avoid missing heart disease cases\")\n",
    "print(\"   - Model can assist in preliminary heart disease screening\")\n",
    "print(\"   - Should be used as decision support, not replacement for doctors\")\n",
    "\n",
    "# Feature insights\n",
    "if hasattr(best_model, 'feature_importances_'):\n",
    "    top_feature = feature_importance.iloc[0]\n",
    "    print(f\"\\n6. Most Important Predictor: {top_feature['feature']} \"\n",
    "          f\"(Importance: {top_feature['importance']:.3f})\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Model Deployment Readiness"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== MODEL DEPLOYMENT ASSESSMENT ===\")\n",
    "\n",
    "deployment_metrics = {\n",
    "    'Criterion': [\n",
    "        'Accuracy > 80%',\n",
    "        'Precision > 75%', \n",
    "        'Recall > 75%',\n",
    "        'Cross-Validation Stability',\n",
    "        'Feature Interpretability',\n",
    "        'Training Time',\n",
    "        'Overall Readiness'\n",
    "    ],\n",
    "    'Status': [\n",
    "        '✅ PASS' if best_accuracy > 0.8 else '⚠️ MARGINAL',\n",
    "        '✅ PASS' if best_precision > 0.75 else '⚠️ MARGINAL',\n",
    "        '✅ PASS' if best_recall > 0.75 else '⚠️ MARGINAL',\n",
    "        '✅ STABLE' if cv_results[best_model_name].std() < 0.05 else '⚠️ VARIABLE',\n",
    "        '✅ GOOD' if hasattr(best_model, 'feature_importances_') else '⚠️ LIMITED',\n",
    "        '✅ FAST' if best_model_name != 'SVM' else '⚠️ SLOW',\n",
    "        '✅ READY' if best_accuracy > 0.8 and best_recall > 0.75 else '⚠️ NEEDS IMPROVEMENT'\n",
    "    ],\n",
    "    'Value': [\n",
    "        f\"{best_accuracy:.1%}\",\n",
    "        f\"{best_precision:.1%}\",\n",
    "        f\"{best_recall:.1%}\",\n",
    "        f\"CV Std: {cv_results[best_model_name].std():.4f}\",\n",
    "        'Interpretable' if hasattr(best_model, 'feature_importances_') else 'Black Box',\n",
    "        '< 1 second',\n",
    "        'Deployable' if best_accuracy > 0.8 and best_recall > 0.75 else 'Needs Tuning'\n",
    "    ]\n",
    "}\n",
    "\n",
    "deployment_df = pd.DataFrame(deployment_metrics)\n",
    "deployment_df"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Save Best Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import joblib\n",
    "\n",
    "# Save the best model\n",
    "model_filename = f'best_heart_disease_model.pkl'\n",
    "joblib.dump(best_model, model_filename)\n",
    "\n",
    "# Save preprocessing objects\n",
    "preprocessing_objects = {\n",
    "    'scaler': scaler,\n",
    "    'imputer': imputer,\n",
    "    'label_encoders': label_encoders,\n",
    "    'feature_names': list(X.columns)\n",
    "}\n",
    "joblib.dump(preprocessing_objects, 'preprocessing_objects.pkl')\n",
    "\n",
    "print(f\"Best model saved as: {model_filename}\")\n",
    "print(\"Preprocessing objects saved as: preprocessing_objects.pkl\")\n",
    "print(\"\\n✅ Model experimentation completed successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 📋 **Summary and Next Steps**\n",
    "\n",
    "### **Key Findings:**\n",
    "- **Best Performing Model**: Identified through comprehensive evaluation\n",
    "- **Performance Metrics**: Achieved clinically relevant accuracy and recall\n",
    "- **Feature Importance**: Understood key predictors of heart disease\n",
    "- **Model Stability**: Verified through cross-validation\n",
    "\n",
    "### **Next Steps:**\n",
    "1. **Model Deployment**: Integrate into clinical decision support system\n",
    "2. **Monitoring**: Implement performance monitoring in production\n",
    "3. **Improvement**: Collect more data for model retraining\n",
    "4. **Validation**: Conduct clinical validation studies\n",
    "\n",
    "### **Clinical Application:**\n",
    "This model can serve as a preliminary screening tool to assist healthcare professionals in identifying patients at risk of heart disease, potentially leading to earlier interventions and improved patient outcomes."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}